In [6]:
pip install konlpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 35.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 41.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import konlpy
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os


In [10]:
#################### 함수 선언
def infinite_loop(browser, sleepDelay):
    # 최초 페이지 스크롤 설정
    # 스크롤 시키지 않았을 때의 전체 높이
    last_page_height = browser.execute_script("return document.documentElement.scrollHeight")

    while True:
        # 윈도우 창을 0에서 위에서 설정한 전체 높이로 이동
        browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(sleepDelay)
        # 스크롤 다운한 만큼의 높이를 신규 높이로 설정 
        new_page_height = browser.execute_script("return document.documentElement.scrollHeight")
        # 직전 페이지 높이와 신규 페이지 높이 비교
        if new_page_height == last_page_height:
            time.sleep(sleepDelay)
            # 신규 페이지 높이가 이전과 동일하면, while문 break
            if new_page_height == browser.execute_script("return document.documentElement.scrollHeight"):
                break
        else:
            last_page_height = new_page_height

def log_check(f, str):
    now = datetime.now()
    f_str = f"{now}    {str}"
    f.write(f_str + "\n")
    print(f_str)

In [12]:
#################### 1. 검색단어, 검색기간 및 기타 파라미터 설정
current_path = os.getcwd()

search_word = "케이카 구매 후기"
start_date = "20181001" #yyyymmdd 형식
end_date  = "20240729" #yyyymmdd 형식

# 출처: 블로그
# 정렬: 관련도순
target_url = f"https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query=%EC%BC%80%EC%9D%B4%EC%B9%B4+%EA%B5%AC%EB%A7%A4+%ED%9B%84%EA%B8%B0"


sleepDelay = 1.5



frist_url_list = [] # 1차로 검색 후 모든 blog에 대해 url을 수집함.


df_text = pd.DataFrame(columns=['date', 'url', 'title', 'text']) 
df_idx = 0
# date: 네이버 블로그 글 생성 날짜
# url: 스크래핑한 url
# title: 스크래핑한 url 내 블로그 제목
# text: 스크래핑한 url 내 블로그 내용


today = f"{datetime.now().year}{datetime.now().month}{datetime.now().day}"
# log파일
f = open(f'{current_path}\\crawler_log_{today}.txt', 'w')
f.write(f"[크롤링 파라미터]" + '\n')
f.write(f"검색 단어: {search_word}" + '\n')
f.write(f"검색 시작일: {start_date}" + '\n')
f.write(f"검색 종료일: {end_date}" + '\n')
f.write("================================================" + '\n')

49

In [14]:
#################### 2. 검색단어, 검색기간을 기준으로 검색 진행 ~ # 3. 검색에 나온 blog url 수집
log_check(f, "크롤링 시작")
browser = webdriver.Chrome()
browser.get(target_url)
browser.implicitly_wait(sleepDelay * 5)

log_check(f, "검색결과 최하단 이동")
infinite_loop(browser, sleepDelay)


log_check(f, "url수집 시작")
url_table = browser.find_elements(By.CLASS_NAME, 'bx')
for i in range(1, len(url_table)):
    if i % 1000 == 0: log_check(f, f"{i}개의 url 수집. 계속해서 수집 진행중")
    try:
        url = browser.find_elements(By.CSS_SELECTOR, f'#sp_blog_{str(i)}')[-1].find_element(By.CLASS_NAME, 'title_link').get_attribute('href')
        frist_url_list.append(url)
    except:
        break

log_check(f, f"url수집 완료. 전체 {len(frist_url_list)}개의 url")
df_ful = pd.DataFrame({'url' : frist_url_list})
df_ful.to_csv(f'{current_path}\\url_list_{today}.csv', index = False, encoding='utf-8-sig') # 크롤링한 url_list
log_check(f, "url csv 생성 완료")
browser.quit()

2024-07-30 16:07:30.055697    크롤링 시작
2024-07-30 16:07:33.411476    검색결과 최하단 이동
2024-07-30 16:07:39.487829    url수집 시작
2024-07-30 16:07:47.022929    url수집 완료. 전체 0개의 url
2024-07-30 16:07:47.038673    url csv 생성 완료


In [16]:
#################### 4. 수집한 blog url을 순회하면서 글생성시간(date), 제목(title), 내용(text)를 수집하여 DataFrame에 저장
log_check(f, "수집한 url 경로에 따라 크롤링 시작")
for idx, url in enumerate(frist_url_list):
    m_url = "https://m." + url.replace("https://","") # //blog.naver 의 경우 크롤링을 막아놔서 모바일 경로인 m.으로 우회
    res = requests.get(m_url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")

    try:
        date = soup.find("p", attrs={'class':'blog_date'}).text.rstrip()
        title = soup.find("span", attrs={'class':'se-fs- se-ff-'}).text
        text = soup.find("div", attrs={'class':'se-main-container'}).text

        df_text.loc[df_idx] = [date, url, title, text]
        df_idx += 1
    except:
        continue

    if (idx + 1) % 1000 == 0: log_check(f, f"{idx + 1}개 url 크롤링 완료. 계속해서 크롤링 진행중")

log_check(f, f"url 크롤링 완료")
log_check(f, f"네이버 블로그 크롤링 완전 종료.")
f.close()

2024-07-30 16:09:11.837908    수집한 url 경로에 따라 크롤링 시작
2024-07-30 16:09:11.838561    url 크롤링 완료
2024-07-30 16:09:11.838602    네이버 블로그 크롤링 완전 종료.


In [24]:
client_id = "h3syfFor_zPWGGMAt0kE"
client_secret = "y51Pjma3v3"


import os
import sys
import urllib.request
import requests

url = "https://openapi.naver.com/v1/search/blog?query=케이카구매후기"
res = requests.get(url, headers = {
    "X-Naver-Client-Id" : client_id, 
    "X-Naver-Client-Secret" : client_secret
    
}  )

result = res.json()


In [26]:
result['total']

3385

In [28]:
result['items']


[{'title': '<b>케이카</b> 중고차 <b>구매</b>조건 더 좋은 카머스 <b>후기</b>',
  'link': 'https://blog.naver.com/woochulssa/223526567428',
  'description': '남편한테도 100% 가서 봐야된다는 마음가짐으로 한 것이긴 하지만 <b>케이카</b> 중고차 홈서비스 <b>후기</b> 보고 걍 집 앞으로 탁송받자고 함. 저희는 목요일에 차를 <b>구매</b>해서 금요일에 받을수있도로 했는데 금요일이라... ',
  'bloggername': '문명의 경향에 달관하여 미래의 추세를 점친다.',
  'bloggerlink': 'blog.naver.com/woochulssa',
  'postdate': '20240726'},
 {'title': '<b>케이카</b>(KCAR) 제네시스 DH(G80) <b>구매후기</b>',
  'link': 'https://blog.naver.com/blade21/223329423795',
  'description': '<b>케이카</b>(KCAR) <b>구매후기</b> 제네시스 DH (G80) <b>구매</b>동기 원래 사용하던 21년식 스팅어를 회사 근무지가 변경됨에 따라 한달 주행거리가 200KM가 되지 않게 되어 판매하게 되었습니다. 한달에 2번정도 주행하는데... ',
  'bloggername': '비가와의 비오는 세상',
  'bloggerlink': 'blog.naver.com/blade21',
  'postdate': '20240121'},
 {'title': '<b>케이카</b> 레이 내차사기 홈서비스 <b>구매</b>방법 <b>후기</b>, 자동차보험... ',
  'link': 'https://blog.naver.com/kjg9288/223387502735',
  'description': 'K5 판매 후 매일매일 틈날 때마다 <b>케이카</b>에서 레이를 찾아보았다 내가 원하는 조건은 1. 가격: 1300만 원... 장착 사진 

In [30]:
len(result['items'])


10

In [32]:
url = "https://openapi.naver.com/v1/search/blog.json?query=케이카구매후기&display=100"
res = requests.get(url, headers = {
    "X-Naver-Client-Id" : client_id, 
    "X-Naver-Client-Secret" : client_secret
    
}  )

result = res.json()

len(result['items'])


100

In [34]:
url = "https://openapi.naver.com/v1/search/blog.json?query=케이카구매후기&start=101&display=101"
res = requests.get(url, headers = {
    "X-Naver-Client-Id" : client_id, 
    "X-Naver-Client-Secret" : client_secret
    
}  )

In [36]:
def call_api(query, start,display):
    url = f"https://openapi.naver.com/v1/search/blog.json?query={query}&start={start}&display={display}"
    res = requests.get(url, headers = {
        "X-Naver-Client-Id" : client_id, 
        "X-Naver-Client-Secret" : client_secret

    }  )
    r = res.json()
    return r
    
r = call_api("케이카구매후기", 1 ,10)

r['items']

[{'title': '<b>케이카</b> 중고차 <b>구매</b>조건 더 좋은 카머스 <b>후기</b>',
  'link': 'https://blog.naver.com/woochulssa/223526567428',
  'description': '남편한테도 100% 가서 봐야된다는 마음가짐으로 한 것이긴 하지만 <b>케이카</b> 중고차 홈서비스 <b>후기</b> 보고 걍 집 앞으로 탁송받자고 함. 저희는 목요일에 차를 <b>구매</b>해서 금요일에 받을수있도로 했는데 금요일이라... ',
  'bloggername': '문명의 경향에 달관하여 미래의 추세를 점친다.',
  'bloggerlink': 'blog.naver.com/woochulssa',
  'postdate': '20240726'},
 {'title': '<b>케이카</b>(KCAR) 제네시스 DH(G80) <b>구매후기</b>',
  'link': 'https://blog.naver.com/blade21/223329423795',
  'description': '<b>케이카</b>(KCAR) <b>구매후기</b> 제네시스 DH (G80) <b>구매</b>동기 원래 사용하던 21년식 스팅어를 회사 근무지가 변경됨에 따라 한달 주행거리가 200KM가 되지 않게 되어 판매하게 되었습니다. 한달에 2번정도 주행하는데... ',
  'bloggername': '비가와의 비오는 세상',
  'bloggerlink': 'blog.naver.com/blade21',
  'postdate': '20240121'},
 {'title': '<b>케이카</b> 레이 내차사기 홈서비스 <b>구매</b>방법 <b>후기</b>, 자동차보험... ',
  'link': 'https://blog.naver.com/kjg9288/223387502735',
  'description': 'K5 판매 후 매일매일 틈날 때마다 <b>케이카</b>에서 레이를 찾아보았다 내가 원하는 조건은 1. 가격: 1300만 원... 장착 사진 

In [46]:
import json
import requests

# API 호출 함수 정의
def call_api(query, start, display):
    url = f"https://openapi.naver.com/v1/search/blog.json?query={query}&start={start}&display={display}"
    res = requests.get(url, headers = {
        "X-Naver-Client-Id" : client_id, 
        "X-Naver-Client-Secret" : client_secret
    })
    r = res.json()
    return r

# API 호출 및 결과 저장
query = "케이카구매후기"
start = 1
display = 10

r = call_api(query, start, display)

# 결과를 JSON 파일로 저장
with open('api_result.json', 'w', encoding='utf-8') as file:
    json.dump(r['items'], file, ensure_ascii=False, indent=4)

print("API 결과가 api_result.json 파일로 저장되었습니다.")


API 결과가 api_result.json 파일로 저장되었습니다.


In [48]:
pip install pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


In [52]:
import json
import requests
import pandas as pd

# API 호출 함수 정의
def call_api(query, start, display):
    url = f"https://openapi.naver.com/v1/search/blog.json?query={query}&start={start}&display={display}"
    res = requests.get(url, headers = {
        "X-Naver-Client-Id" : client_id, 
        "X-Naver-Client-Secret" : client_secret
    })
    r = res.json()
    return r

# API 호출 및 결과 저장
query = "케이카구매후기"
start = 1
display = 10

r = call_api(query, start, display)

# 결과를 DataFrame으로 변환
df = pd.DataFrame(r['items'])

# DataFrame을 엑셀 파일로 저장
df.to_excel('api_result.xlsx', index=False)

print("API 결과가 api_result.xlsx 파일로 저장되었습니다.")


API 결과가 api_result.xlsx 파일로 저장되었습니다.


In [58]:
urls = [
    "https://blog.naver.com/woochulssa/223526567428",
    "https://blog.naver.com/blade21/223329423795",
    "https://blog.naver.com/kjg9288/223387502735",
    "https://blog.naver.com/topann/222805047002",
    "https://blog.naver.com/koojawongtr/223320822417",
    "https://blog.naver.com/d__yng/223462076369",
    "https://blog.naver.com/inyo1127/223525315091",
    "https://blog.naver.com/fear666/223527469861",
    "https://blog.naver.com/dsw99248/223451274252",
    "https://blog.naver.com/ggom_nyang/223455179389"
]
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_blog_content(url):
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # 블로그 본문 추출
    content = soup.select_one('.se-main-container')
    
    if content:
        return content.get_text(separator='\n').strip()
    else:
        return "본문을 찾을 수 없습니다."

# 추출한 URL에 접근하여 본문 크롤링
blog_contents = []

for url in urls:
    content = get_blog_content(url)
    blog_contents.append(content)

# 결과를 엑셀 파일로 저장
df = pd.DataFrame({'URL': urls, 'Content': blog_contents})
df.to_excel('blog_contents.xlsx', index=False)

print("블로그 본문이 blog_contents.xlsx 파일로 저장되었습니다.")


블로그 본문이 blog_contents.xlsx 파일로 저장되었습니다.


In [60]:
pip install selenium webdriver-manager pandas


Note: you may need to restart the kernel to use updated packages.


In [68]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# URL 리스트
urls = [
    "https://blog.naver.com/woochulssa/223526567428",
    "https://blog.naver.com/blade21/223329423795",
    "https://blog.naver.com/kjg9288/223387502735",
    "https://blog.naver.com/topann/222805047002",
    "https://blog.naver.com/koojawongtr/223320822417",
    "https://blog.naver.com/d__yng/223462076369",
    "https://blog.naver.com/inyo1127/223525315091",
    "https://blog.naver.com/fear666/223527469861",
    "https://blog.naver.com/dsw99248/223451274252",
    "https://blog.naver.com/ggom_nyang/223455179389"
]

# Selenium WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 브라우저를 열지 않음
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# WebDriver 초기화
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

def get_blog_content(url):
    driver.get(url)
    time.sleep(3)  # 페이지 로딩을 기다림
    
    try:
        # 동적 콘텐츠 로드 대기 및 본문 추출
        content = driver.find_element(By.CSS_SELECTOR, '.se-main-container').text
        return content
    except Exception as e:
        return f"본문을 찾을 수 없습니다. 오류: {str(e)}"

# 추출한 URL에 접근하여 본문 크롤링
blog_contents = []

for url in urls:
    content = get_blog_content(url)
    blog_contents.append(content)

# WebDriver 종료
driver.quit()

# 결과를 엑셀 파일로 저장
df = pd.DataFrame({'URL': urls, 'Content': blog_contents})
df.to_excel('blog_contents.xlsx', index=False)

print("블로그 본문이 blog_contents.xlsx 파일로 저장되었습니다.")


블로그 본문이 blog_contents.xlsx 파일로 저장되었습니다.


In [64]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_blog_content(url):
    driver.get(url)
    try:
        # 최대 10초 대기하며 특정 요소가 로드될 때까지 기다립니다.
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.se-main-container'))
        )
        content = driver.find_element(By.CSS_SELECTOR, '.se-main-container').text
        return content
    except Exception as e:
        return f"본문을 찾을 수 없습니다. 오류: {str(e)}"


In [66]:
def get_blog_content(url):
    driver.get(url)
    time.sleep(3)
    page_source = driver.page_source  # 전체 페이지 소스를 가져옴
    print(page_source)  # 디버깅용으로 전체 HTML 출력
    try:
        content = driver.find_element(By.CSS_SELECTOR, '.se-main-container').text
        return content
    except Exception as e:
        return f"본문을 찾을 수 없습니다. 오류: {str(e)}"


In [78]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# URL 리스트
urls = [
    "https://blog.naver.com/woochulssa/223526567428",
    "https://blog.naver.com/blade21/223329423795",
    "https://blog.naver.com/kjg9288/223387502735",
    "https://blog.naver.com/topann/222805047002",
    "https://blog.naver.com/koojawongtr/223320822417",
    "https://blog.naver.com/d__yng/223462076369",
    "https://blog.naver.com/inyo1127/223525315091",
    "https://blog.naver.com/fear666/223527469861",
    "https://blog.naver.com/dsw99248/223451274252",
    "https://blog.naver.com/ggom_nyang/223455179389"
]

# Selenium WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 브라우저를 열지 않음
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# WebDriver 초기화
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

def get_blog_content(url):
    driver.get(url)
    
    try:
        # 최대 10초 대기하며 본문 요소가 로드될 때까지 기다립니다.
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.se-main-container'))
        )
        # 본문 콘텐츠를 추출합니다.
        content = driver.find_element(By.CSS_SELECTOR, '.se-main-container').text
        return content
    except Exception as e:
        return f"본문을 찾을 수 없습니다. 오류: {str(e)}"

# 추출한 URL에 접근하여 본문 크롤링
blog_contents = []

for url in urls:
    content = get_blog_content(url)
    blog_contents.append(content)

# WebDriver 종료
driver.quit()

# 결과를 엑셀 파일로 저장
df = pd.DataFrame({'URL': urls, 'Content': blog_contents})
df.to_excel('blog_contents.xlsx', index=False)

print("블로그 본문이 blog_contents.xlsx 파일로 저장되었습니다.")


블로그 본문이 blog_contents.xlsx 파일로 저장되었습니다.


In [76]:
def get_blog_content(url):
    driver.get(url)
    time.sleep(5)  # 페이지 로딩을 기다림
    page_source = driver.page_source  # 전체 페이지 소스를 가져옴
    print(page_source)  # 디버깅용으로 전체 HTML 출력
    try:
        content = driver.find_element(By.CSS_SELECTOR, '.se-main-container').text
        return content
    except Exception as e:
        return f"본문을 찾을 수 없습니다. 오류: {str(e)}"
